In [ ]:
import random
import numpy as np
import copy as cp
from scipy.special import comb


class Board:
    def __init__(self, n):
        self.n_queen = n
        self.map = [[0 for j in range(n)] for i in range(n)]


    def get_dim(self) :
      return self.n_queen


    def get_queen(self, r) :
      '''
      Retrieves column position of queen in row 'r' 
      '''
      c = 0
      #
      for e in self.map[r] :
        if e == 1 :
          return c
        else :
          c += 1


    def queen_on(self, r, c) :
      '''
      Place a queen at row 'r' column 'c'
      '''
      self.map[r][c] = 1

  
    def queen_off(self, r, c) :
      '''
      Remove queen from row 'r' column 'c'
      '''
      self.map[r][c] = 0


    def set_queens(self) :
        for i in range(self.n_queen):
            j = random.randint(0, self.n_queen - 1)
            self.map[i][j] = 1
 

    def reset_row(self, r) :
      '''
      Move queen to column 0 in row 'r'
      '''
      self.map[r][self.get_queen(r)] = 0
      self.map[r][0] = 1


    def fitness(self):
        fit = self.n_queen * (self.n_queen - 1) // 2
        for i in range(self.n_queen):
            for j in range(self.n_queen):
                if self.map[i][j] == 1:
                    for k in range(1, self.n_queen - i):
                        if self.map[i + k][j] == 1:
                            fit -= 1
                        if j - k >= 0 and self.map[i + k][j - k] == 1:
                            fit -= 1
                        if j + k < self.n_queen and self.map[i + k][j + k] == 1:
                            fit -= 1
        return fit


    def show(self):
        print(np.matrix(self.map))
        print( "Fitness: ",  self.fitness(),"\n" )
#
#
#       
if __name__ == '__main__':
    test = Board(5)
    test.set_queens()
    test.show()    

In [ ]:
#
#
#
class Hill_Climb:
    '''
    @param b : pre-initialized Board object to be optimized
    @param n : max number of allowed steps
    '''
    def __init__(self, b, n) :
        self.step = 0
        self.board = b
        self.max_steps = n


    def optimize(self) :
        '''
        Attempts to find a solution within "self.max_steps" steps
        '''
        best_fit = self.board.fitness()
        best_clmns = []
        # initialize optimal solution to original config
        for r in range( self.board.get_dim() ) :
            best_clmns.append( self.board.get_queen(r) )
        # loop until soln found or max_steps taken
        while ( best_fit < comb(self.board.get_dim(), 2, exact=True) ) and (self.step < self.max_steps) :
            # step through each row
            for r in range( self.board.get_dim() ) :
                # memorize original column position
                pos0 = cp.deepcopy( best_clmns[r] )
                # pick queen up
                self.board.queen_off(r, self.board.get_queen(r) )
                # slide queen along row
                for c in range( self.board.get_dim() ) :
                    # put queen down
                    self.board.queen_on(r, c)
                    if self.board.fitness() == comb(self.board.get_dim(), 2, exact=True) :
                        best_clmns[r] = c
                        self.step += 1
                        return 
                    elif self.board.fitness() > best_fit :
                        # save column with better fit
                        best_clmns[r] = c
                        best_fit = self.board.fitness()
                    # pick queen up  
                    self.board.queen_off(r, c)
                # return queen to best column before moving to next row
                self.board.queen_on( r, best_clmns[r] )
                self.step += 1  
                print("STEP:", self.step)
                self.board.show()


    def show(self) :
        self.board.show()     
        print("\nSteps:", self.step)  
#
#
#
if __name__ == '__main__':
    board = Board(5)
    board.set_queens()
    board.fitness()
    print("INITIAL CONFIGURATION")
    board.show() 

    climb = Hill_Climb(board, 10000)
    climb.optimize()
    print("FINAL CONFIGURATION")
    climb.show()